In [1]:
import numpy as np
import scipy

### 1. (a) Inverse via Gauss-Jordan

In [2]:
# Compute the inverse of a matrix via Gauss-Jordan (with no pivoting)
def inv(A):
  n = len(A)
  A_aug = np.concatenate((A, np.identity(n)), axis=1)

  for i in range(n):
    scale = A_aug[i][i]
    for j in range(2 * n):
      A_aug[i][j] /= scale
    for k in range(n):
      scale2 = A_aug[k][i]
      if k != i:
        for j in range(2 * n):
          A_aug[k][j] -= scale2 * A_aug[i][j]
  return A_aug[:, n:]

In [3]:
n = 3

A = np.random.rand(n, n)
A_inv = inv(A)

# We expect something close the the identity matrix
print(A @ A_inv)

[[ 1.00000000e+00 -3.80490138e-15  1.32735205e-14]
 [ 1.69274405e-16  1.00000000e+00  1.83041481e-15]
 [ 4.84647097e-17 -3.67012229e-16  1.00000000e+00]]


### 1. (b) Inverse via LU

In [4]:
# Compute the inverse of a lower triangular matrix
def tril_inv(L):
  n = len(L)
  L_inv = np.zeros((n, n))

  for i in range(n):
    L_inv[i][i] = 1 / L[i][i]
    for j in range(i):
      dot = 0
      for k in range(j, i):
        dot -= L[i][k] * L_inv[k][j]
      dot /= L[i][i]
      L_inv[i][j] = dot
  return L_inv

In [5]:
# Compute the inverse of an upper triangular matrix
def triu_inv(U):
  return tril_inv(U.T).T

In [6]:
# Multiply upper and lower triangular matrices
def tri_matmul(U, L):
  n = len(U)
  A = np.zeros((n, n))

  for i in range(n):
    for j in range(n):
      dot = 0
      for k in range(max(i, j), n):
        dot += U[i][k] * L[k][j]
      A[i][j] = dot
  return A

In [7]:
# Compute the inverse of an arbitrary matrix using the above
def inv_lu(A):
  _, L, U = scipy.linalg.lu(A)
  return tri_matmul(triu_inv(U), tril_inv(L))

In [8]:
n = 4

A = np.random.rand(n, n)
A_inv = inv_lu(A)

# Again, we expect a (likely permuted) identity matrix
print(A @ A_inv)

[[-1.24265267e-16  3.68600672e-16  1.00000000e+00  2.28318536e-17]
 [-6.62225172e-16  1.00000000e+00  4.02138904e-16  4.27819125e-16]
 [ 1.00000000e+00 -1.06393104e-15  5.90768867e-16  7.73417831e-16]
 [-7.81485215e-16 -6.06928075e-16  5.27345596e-16  1.00000000e+00]]


### 3. (a) Cholesky Algorithm

In [9]:
# Compute Cholesky decomposition; terminate on non-positve diagonal entries
def cholesky(A):
  n = len(A)
  L = np.zeros((n, n))

  for j in range(n):
    L[j][j] = A[j][j]
    for k in range(j):
      L[j][j] -= L[j][k] ** 2
    if L[j][j] <= 0:
      print("The matrix is not positive definite")
      return None
    L[j][j] = np.sqrt(L[j][j])
    for i in range(j + 1, n):
      L[i][j] = A[i][j]
      for k in range(j):
        L[i][j] -= L[i][k] * L[j][k]
      L[i][j] /= L[j][j]
  return L

### 3. (b) Cholesky for $\tilde{A} + \tilde{A}^\top$

In [10]:
At = np.random.rand(100, 100)
A = At + At.T

# We do NOT expect A to be symmetric positive definite
L = cholesky(A)

The matrix is not positive definite


In [11]:
# Provided minimal eigenvalue <= 0, our conclusion is correct
evals, evecs = np.linalg.eig(A)
print(np.min(evals))

-7.9757142895093205


### 3. (c) Cholesky for $\tilde{A}^\top \tilde{A}$

In [12]:
At = np.random.rand(100, 100)
A = At.T @ At

# We expect A to be symmetric positive definite
L = cholesky(A)

In [13]:
# Compare norms of Cholesky with my command and standard one
L_std = np.linalg.cholesky(A)
print(np.linalg.norm(L - L_std))

2.996038104490378e-12
